In [1]:
!pip3 install lxml
import requests
import pandas as pd
import lxml.etree as et


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
books = []
author = []
texts = []
rating = []

In [3]:
def init(num):
    str_ind = str(num)
    source = requests.get("https://www.chitai-gorod.ru/catalog/books/klassicheskaya-proza-110003?page=" + str_ind)
    initial_root = et.fromstring(source.content, parser=et.HTMLParser(remove_comments=True))
    return initial_root

In [4]:
def parse_description(root_of_html):
   messages = root_of_html.xpath("//div[@class='product-reviews__cards']")
   for m in messages:
      message = m.xpath("//div[@class='product-review-card__comment']")
      for new_m in message:
          texts.append(new_m.text)     
          all_content = root_of_html.xpath("//div[@class='product-detail-title']")
          for a in all_content:
              books.append(a.xpath("div/h1")[0].text.replace("\n", ""))
              if len(a.xpath("div/div")) == 1:
                  author.append(a.xpath("//div[@class='product-detail-title__authors']/a")[0].text.replace("\n", ""))
              else:
                   author.append(None)

In [5]:
def stars(root_of_html):
    rating_stars = root_of_html.xpath("//div[@class='product-review-card__rating']")
    for r in rating_stars:
        num_of_stars = int(r.xpath("span[@class='box-none']")[0].text.replace("\n", ""))
        if num_of_stars == 3:
            rating.append("Neutral")
        if num_of_stars < 3:
            rating.append("Negative")
        if num_of_stars > 3:
            rating.append("Positive")

In [6]:
def parse_inside(root_of_html):
    url = root_of_html.xpath("//article/a/@href")
    urls = []
    ind = 0
    for u in url:
        if ind % 2 == 0:
            urls.append("https://www.chitai-gorod.ru" + u)
        ind = ind + 1   
    for j in range(0, len(urls)):
        source2 = requests.get(urls[j])
        root2 = et.fromstring(source2.content, parser=et.HTMLParser(remove_comments=True))
        parse_description(root2)
        stars(root2)

In [7]:
for index in range(1, 9):
    root= init(1)
    parse_inside(root)

In [8]:
dictionary = { 'book_title': books, 'author_of_the_book': author, 'texts': texts, 'sentiment': rating }
dataset = pd.DataFrame.from_records(dictionary)
dataset.to_csv('texts_dataset.csv', sep=' ')
pd.read_csv('texts_dataset.csv', delimiter=' ')
dataset

,author_of_the_book,book_title,sentiment,texts
0,Цао Сюэ-цинь,Сон в красном тереме. Том 1. Том 2 (ко...,Positive,"Наконец-то Азбука выпустила этот роман, так ка..."
1,Цао Сюэ-цинь,Сон в красном тереме. Том 1. Том 2 (ко...,Positive,Издательство Иностранка порадовало переиздание...
2,Цао Сюэ-цинь,Сон в красном тереме. Том 1. Том 2 (ко...,Positive,Никогда раньше ничего не слышала о таком произ...
3,Оскар Уайльд,Портрет Дориана Грея,Negative,"Наверное, самое переоцененное произведение сов..."
4,Оскар Уайльд,Портрет Дориана Грея,Positive,"Оскар Уайльд - отличный английский философ, вс..."
...,...,...,...,...
1018,Федор Достоевский,Белые ночи,Negative,Совершенно не такой сюжет себе представляла пр...
1019,Федор Достоевский,Белые ночи,Positive,"Очень приятные произведения, меланхоличные, за..."
1020,Михаил Булгаков,Морфий,Positive,Как-то в школе писала исследовательскую работу...
1021,Михаил Булгаков,Морфий,Positive,Булгаков -гений!
